### https://riptutorial.com/julia-lang/example/20459/interpolation-and-expressions

In [1]:
n = 2
MyString = "there are $n ducks"

"there are 2 ducks"

We use the `$` sign to insert the value of `n` into the string. We can use the same technique with expressions. E.g.<br>
使用 `$` 符号将 `n` 的值插入到字符串中。我们可以用同样的技巧处理表达式。如。

In [3]:
a = 2
ex1 = :(x = 2*$a)
ex1 |> display

a = 3
ex1 |> display
eval(ex1)

:(x = 2 * 2)

:(x = 2 * 2)

4

In [4]:
a = 2
ex2 = :(x = 2*a)
ex2 |> display

a = 3
eval(ex2)

:(x = 2a)

6

Thus, with the first example, we set in advance the value of `a` that will be used at the time that the expression is evaluated. With the second example, however, the Julia compiler will only look to `a` to find its value at the time of evaluation for our expression.<br>

因此，在第一个示例中，我们提前设置了 `a` 的值，该值将在计算表达式时使用。然而，对于第二个示例，Julia编译器将只查找 `a` 来查找表达式求值时的值。

### https://docs.juliacn.com/latest/manual/metaprogramming/#man-expression-interpolation

使用值参数直接构造 Expr 对象虽然很强大，但与「通常的」 Julia 语法相比，Expr 构造函数可能让人觉得乏味。作为替代方法，Julia 允许将**字面量**或**表达式**插入到**被引用的表达式**中。表达式插值由前缀 $ 表示。

In [1]:
a = 1
ex1 = :($a + b)

:(1 + b)

**对未被引用的表达式进行插值是不支持的**，这会导致编译期错误：

In [1]:
$a + b

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X12sZmlsZQ==.jl:1

下例中，元组 (1,2,3) 作为表达式插入到条件测试中：

In [3]:
ex2 = :(a in $:((1,2,3)) )
# $ 插入变量 插入表达式，这里是插入表达式 :((1,2,3)), 表达式拼接去掉:

:(a in (1, 2, 3))

In [3]:
:(a in $(1,2,3) )
# 这个是插入变量

:(a in (1, 2, 3))

In [5]:
:(3 > $:( 1+1 ) ) |> display # 插入拼接表达式
:(3 > $(1+1) ) |> display # 相当于插入变量

:(3 > 1 + 1)

:(3 > 2)

In [5]:
ex_interploted = :((1,2,3))
ex_interploted |> display
ex_interploted |> typeof |> display
eval(ex_interploted)

:((1, 2, 3))

Expr

(1, 2, 3)

In [6]:
:(a in $ex_interploted )

:(a in (1, 2, 3))

In [7]:
Meta.show_sexpr(ex_interploted)

(:tuple, 1, 2, 3)

In [8]:
dump(ex_interploted)

Expr
  head: Symbol tuple
  args: Array{Any}((3,))
    1: Int64 1
    2: Int64 2
    3: Int64 3


In [9]:
Expr(:tuple, 1, 2, 3)

:((1, 2, 3))

**Symbol的值,即eval, 为其对应的变量, eval(:x)=x**<br><br>
Splatting 插值<br>
表达式组成的数组，需要它们都变成其所处表达式的参数，可通过 **$(xs...)** 做到。例如，下面的代码生成一个函数调用，其**参数数量通过编程确定：**

In [11]:
args = [:x, :y, :z];
:(f(1, $(args...)))

:(f(1, x, y, z))

### https://docs.juliacn.com/latest/manual/metaprogramming/#嵌套引用

In [2]:
Expr(:call,:+,1,2) |> display
quote_dollar_Expr_end =
quote
    $(Expr(:call,:+,1,2)) 
end

:(1 + 2)

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:4 =#
    1 + 2
end

In [3]:
quote_Expr_end =
quote
    Expr(:call,:+,1,2)
end

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:3 =#
    Expr(:call, :+, 1, 2)
end

In [4]:
Base.remove_linenums!(quote_dollar_Expr_end)
Base.remove_linenums!(quote_Expr_end)
(quote_dollar_Expr_end == quote_Expr_end) |> display

eval(quote_dollar_Expr_end) |> display
eval(quote_Expr_end) |> display

false

3

:(1 + 2)

In [6]:
Meta.show_sexpr(quote_dollar_Expr_end)

(:block,
  (:call, :+, 1, 2)
)

In [11]:
quote_dollar_Expr_end2 = quote :(1+2) end
Base.remove_linenums!(quote_dollar_Expr_end2)

quote
    $(Expr(:quote, :(1 + 2)))
end

In [14]:
Meta.show_sexpr(quote_dollar_Expr_end2)
println("\n")
dump(quote_dollar_Expr_end2)

(:block,
  (:quote, (:call, :+, 1, 2))
)

Expr
  head: Symbol block
  args: Array{Any}((1,))
    1: Expr
      head: Symbol quote
      args: Array{Any}((1,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Int64 1
            3: Int64 2


In [16]:
quote_dollar_Expr_end2 ==
quote
    $(
        Expr(:quote,
             $(Expr(:call, :+, 1, 2))
            )
    )
end

ErrorException: syntax: "$" expression outside quote around e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X54sZmlsZQ==.jl:1

In [17]:
quote_dollar_Expr_end2 ==
quote
    $(
        Expr(:quote,
             :($(Expr(:call, :+, 1, 2)))
            )
    )
end

false

In [22]:
eval(quote_dollar_Expr_end2) |> display
eval(quote_dollar_Expr_end2) == eval(
quote
    $(
        Expr(:quote,
             :($(Expr(:call, :+, 1, 2)))
            )  # Expr(:quote,ex), ex是表达式形式,并未处于引用quote中
    )
end
)

:(1 + 2)

true

### https://riptutorial.com/julia-lang/example/26313/guide

In [20]:
one_plus_one = Symbol("1 + 1")

Symbol("1 + 1")

In [21]:
eval(one_plus_one)

UndefVarError: UndefVarError: `1 + 1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [22]:
eval(Symbol("1 + 1"))

UndefVarError: UndefVarError: `1 + 1` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [23]:
valid_math = :($one_plus_one = 3)

:(var"1 + 1" = 3)

In [24]:
one_plus_one_plus_two = :($one_plus_one + 2)

:(var"1 + 1" + 2)

In [25]:
ex4= quote
    $valid_math
    $one_plus_one_plus_two
    @show($one_plus_one_plus_two)
end

quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:2 =#
    var"1 + 1" = 3
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:3 =#
    var"1 + 1" + 2
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:4 =#
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:4 =# @show var"1 + 1" + 2
end

In [26]:
Base.remove_linenums!(ex4)

quote
    var"1 + 1" = 3
    var"1 + 1" + 2
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:4 =# @show var"1 + 1" + 2
end

这里能看出来, `quote ... end`, `:(statements)` 包裹起来成为`block`, eval的时候把所包裹的逐一执行

In [27]:
Meta.show_sexpr(ex4)

(:block,
  (:(=), Symbol("1 + 1"), 3),
  (:call, :+, Symbol("1 + 1"), 2),
  (:macrocall, Symbol("@show"), :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl:4 =#), (:call, :+, Symbol("1 + 1"), 2))
)

In [29]:
dump(ex4)

Expr
  head: Symbol block
  args: Array{Any}((3,))
    1: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol 1 + 1
        2: Int64 3
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol 1 + 1
        3: Int64 2
    3: Expr
      head: Symbol macrocall
      args: Array{Any}((3,))
        1: Symbol @show
        2: LineNumberNode
          line: Int64 4
          file: Symbol e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X31sZmlsZQ==.jl
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Symbol 1 + 1
            3: Int64 2


In [28]:
eval(
quote
    $valid_math
    $one_plus_one_plus_two
    @show($one_plus_one_plus_two)
end
)

var"1 + 1" + 2 = 5


5

### https://riptutorial.com/julia-lang/example/26313/guide

`:(foo) `means "don't look at the value, look at the expression" `$foo` means "change the expression to its value"<br>
`:(foo) `的意思是“不看值，看表达式”  `$foo` 表示“将表达式更改为其值”<br><br>

`:($(foo)) == foo`. `$(:(foo))` is an error. `$(...) `isn't an operation and doesn't do anything by itself, it's an "interpolate this!" sign that the quoting syntax uses. i.e. **\$(...) only exists within a quote.**<br>
`:($(foo)) == foo` 。  `$(:(foo))` 为错误。 ` $(...)` 不是一个操作，它本身不做任何事情，它是引号语法使用的“插入这个！”标志。也就是说，它只存在于引号中。

In [45]:
foo = 1
Meta.show_sexpr(:($(foo)))

println("\n")
@show :($(foo)) == foo

1

$(Expr(:quote, :($(Expr(:$, :foo))))) == foo = true


true

In [46]:
foo = 1+1
Meta.show_sexpr(:($(foo)))

println("\n")
@show :($(foo)) == foo

2

$(Expr(:quote, :($(Expr(:$, :foo))))) == foo = true


true

In [47]:
dump(:($(foo)))

Int64 2


`$foo` is exchanged for the compile-time value, `eval(foo)` means to do that at runtime<br>
`$foo` 被交换为编译时值  `eval(foo)` 表示在运行时执行<br><br>

eval will occur in the global scope,interpolation $ is local<br>
eval 将在全局作用域中出现 插值是局部的<br><br>

`eval(:<expr>)` should return the same as just `<expr>` (assuming `<expr>` is a valid expression in the current global space)<br>
`eval(:<expr>)` 应该返回与 `<expr>` 相同的值（假设 `<expr>` 在当前全局空间中是一个有效的表达式）

In [34]:
@show eval(:(1 + 2)) == 1 + 2
Meta.show_sexpr(:(1 + 2))
eval(:(1 + 2))

eval($(Expr(:quote, :(1 + 2)))) == 1 + 2 = true
(:call, :+, 1, 2)

3

In [37]:
@show eval(:(let x=1; x + 1 end)) == let x=1; x + 1 end
println("\n")

Meta.show_sexpr(:(let x=1; x + 1 end))
eval(:(let x=1; x + 1 end))

eval($(Expr(:quote, :(let x = 1
      #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:1 =#
      x + 1
  end)))) == let x = 1
        #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:1 =#
        x + 1
    end = true


(:let, (:(=), :x, 1), (:block,
    :(#= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X45sZmlsZQ==.jl:3 =#),
    (:call, :+, :x, 1)
  ))

2

### https://discourse.julialang.org/t/macro-hygiene-for-macro-m-ex/80859

In [ ]:
macro m1(ex)
    :($ex)
end